In [ ]:
import pandas as pd
import numpy as np
from pyknp import Juman
from pandas_ods_reader import read_ods
import re
#reference: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
#         : https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

#read the 5 point data
df5 = read_ods(file_or_path ="dt.ods", sheet = 2)
comments_data5 = df5.iloc[0:,8]
comments_data5.index = range(len(comments_data5))


In [ ]:
len(comments_data5)
###
#for i in comments_data5:
#    i = i.split("。")
#    print(i)
#    text=re.split(r'( ^`^b|！|？)\s*',i,maxsplit=10)
#    for m in text:
#       final = re.split(r'。',m)
#       for j in final:
#           if len(j) >=2:
#               df.append(j)
#df = pd.Series(df)
#comments_data5=df
#print(len(comments_data5))
#print(comments_data5[0:5])
###

In [ ]:
# wakeru
juman = Juman()
def wakeru(x):
#    print(x)
    result = juman.analysis(x)
#for m in mrphs:
#    print(m.midasi, m.genkei, m.hinsi, m.bunrui, m.katuyou2)
    a = []
    for mrph in result.mrph_list():
        if mrph.hinsi =="名詞" or mrph.hinsi == "形容詞" or mrph.hinsi =="動詞":
            if mrph.genkei != None:
                a.append(mrph.genkei)
    return a
comment_pos = comments_data5.apply(wakeru)
#print(comment[0:5])
# read stopwords
stopword_path = "Japanese.txt"
st = pd.read_csv(stopword_path, encoding = 'utf-8', header = None)
#print(comment[48]) #test
stop_w = st.values.tolist()
#print(type(stop_w)) test
stw = list()
for i in stop_w:
    for j in i:
        stw.append(j)
#print(stw) #test
#print('あそこ' not in stw) test
a=0
# delete stopwords
for m in range(0,20):
    for i in comment_pos:
        for j in i:
            if j not in stw:
                j=j
            else:
                i.remove(j)
len(comment_pos)

print(comment_pos[0:3])
print("--------------------------------")
print(comments_data5[0:3])

print(comment_pos[0])
print(comments_data5[0])

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models

#Build LDA model
#reference: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
#         : https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0
#Build dictionary
pos_dict = corpora.Dictionary(comment_pos)
pos_corpus = [pos_dict.doc2bow(i) for i in comment_pos]
#bag of word

In [ ]:
#corpus=pos_corpus, texts=comments_data5
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
    coherence_values = []
    model_list = []
    log_per=[]
    for num_topics in range(start, limit, step):
        model =  models.LdaModel(corpus = corpus,
                                  num_topics = num_topics,
                                  id2word = pos_dict,
                                  random_state = 30,
                                  chunksize = 300,
                                  passes = 10,
                                  alpha = 'auto',
                                  per_word_topics = True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=comment_pos, dictionary=pos_dict, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        log_per.append(model.log_perplexity(pos_corpus))
    return model_list, coherence_values, log_per

In [ ]:
model_list, coherence_values, log_per = compute_coherence_values(dictionary=pos_dict, corpus=pos_corpus, texts=comments_data5, start=2, limit=30, step=1)

In [ ]:
# Show graph
# the bigger the better
import matplotlib.pyplot as plt
limit=30; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.savefig("Coherence")
plt.show()

In [ ]:
# the smaller the better
limit=30; start=2; step=1;
x = range(start, limit, step)
per = []
for i in log_per:
    per.append(np.exp2(-i))
plt.plot(x, per)
plt.xlabel("Num Topics")
plt.ylabel("Perplexity")
plt.legend(("Perplexity"), loc='best')
plt.savefig("Perplexity")
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
for m, cv in zip(x, per):
    print("Num Topics =", m, " has Perplexity Value of", round(cv, 4))    

In [ ]:
#select a topic nums
num_topics_pos=3
tfidf = models.TfidfModel(pos_corpus)
corpus_tfidf = tfidf[pos_corpus]

#pos_corpus = corpus_tfidf
#lda
pos_lda = models.LdaModel(corpus = pos_corpus,
                          num_topics = num_topics_pos,
                          id2word = pos_dict,
                          random_state = 30,
                          chunksize = 300,
                          passes = 10,
                          alpha = 'auto',
                          per_word_topics = True) 
for i in range(num_topics_pos):
    print('pos_topic' + str(i))
    print(pos_lda.print_topic(i))


print('\nPerplexity: ', pos_lda.log_perplexity(pos_corpus))

coherence_model_lda = CoherenceModel(model=pos_lda, texts=comment_pos, dictionary=pos_dict, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt
#from sklearn.manifold import MDS, TSNE
vis = pyLDAvis.gensim.prepare(pos_lda, pos_corpus, pos_dict)
pyLDAvis.save_html(vis,"lda.html")
pyLDAvis.display(vis)

In [ ]:
def format_topics_sentences(ldamodel=pos_lda, corpus=pos_corpus, texts=comments_data5):
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
    # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0: # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=pos_lda, corpus=pos_corpus, texts=comments_data5)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)



In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(5)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]
sent_topics_sorteddf_mallet.to_csv('good1.csv')
# Show
sent_topics_sorteddf_mallet

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)


# Concatenate Column wise
df_dominant_topics = pd.concat([topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = [ 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics